In [0]:
!pip install feedparser
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
import feedparser
from selenium import webdriver
import pandas as pd
import urllib.request
import re
from bs4 import BeautifulSoup

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=chrome_options)



     |████████████████████████████████| 194kB 2.8MB/s 
  Created wheel for feedparser: filename=feedparser-5.2.1-cp36-none-any.whl size=44940 sha256=da16e2385f2421b83086f542dcd1208192d9f49bf4f74f5c476ca133b299c04b
  Stored in directory: /root/.cache/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
Successfully built feedparser
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 25 not upgraded.
Need to get 74.4 MB of archives.
After this operation, 264 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.c

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()


In [0]:
NewsFeed = feedparser.parse("https://rss.cbc.ca/lineup/health.xml/")

print('Number of RSS posts :', len(NewsFeed.entries))

entry = NewsFeed.entries



NameError: ignored

In [0]:
wd.get("https://www.ontario.ca/page/2019-novel-coronavirus")
tables = wd.find_elements_by_tag_name("table")

In [0]:
def find_table(wd):
    return wd.find_elements_by_tag_name("table")

def find_rows(table, skiprow=-1):
    rows = []
    element_lists = table.find_elements_by_tag_name("tr")
    for i in range(len(element_lists)):
        #  if skiprow >= i:
        #      continue
         rows.append(element_lists[i])
    return rows

def find_cell(row):
    cells = []
    for i in row.find_elements_by_tag_name("td"):
        cells.append(i.text)
    return cells

def covnvert_to_df (wd, number=1):
    tables = find_table(wd)
    table = tables[number]
    rows = find_rows(table)
    data = []
    columns = ["Case number","Patient (age and gender)", "Public Health Unit", "Transmission (community, travel or close contact)","Status (self-isolating or hospitalized)"]
# "Hospital (if applicable)"
    for i in range(len(rows)):
        data.append(find_cell(rows[i]))
    df = pd.DataFrame.from_records(data)
    df = df.drop(index=0)
    df.columns = columns
    df.index = df['Case number']
    df = df.drop(columns=['Case number'])
    return df

def filter_df(df):
    df = df[(df['Public Health Unit'] =='Toronto')|(df['Public Health Unit'] == 'Peel')\
    |(df['Public Health Unit'] == 'York')|(df['Public Health Unit'] == 'Pending')\
    |(df['Public Health Unit'] == 'Durham')|(df['Public Health Unit'] == 'Halton')]
    return df

In [0]:
df = covnvert_to_df(wd)
df = filter_df(df)

In [0]:
df2 = covnvert_to_df(wd, number=2)
df2 = filter_df(df2)

In [0]:
df

,Patient (age and gender),Public Health Unit,"Transmission (community, travel or close contact)",Status (self-isolating or hospitalized)
Case number,,,,
218,50s Female,Halton,"travel (Spain, Finland)",self-isolating
219,50s Male,Halton,"travel (USA, San Juan)",self-isolating
220,50s Female,Peel,"travel (cruise), close contact",pending
222,20s Male,Toronto,travel (England),self-isolating
224,20s Male,Toronto,pending,self-isolating
226,40s Male,Toronto,pending,pending
227,30s Female,Toronto,pending,pending
228,20s Female,Toronto,pending,pending
229,20s Male,Toronto,pending,pending


In [0]:
df2

,Patient (age and gender),Public Health Unit,"Transmission (community, travel or close contact)",Status (self-isolating or hospitalized)
Case number,,,,
258,50s Male,Halton,pending,deceased


In [0]:
df.to_csv("Table1March19")
df2.to_csv("Table2March19")

In [0]:
entry = feedparser.parse("http://toronto.ctvnews.ca/rss/Latest").entries
for article in entry:
    print(article)

{'title': 'Toronto mayor goes into self-isolation after returning from U.K.', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://toronto.ctvnews.ca/rss/ctv-news-toronto-1.822319', 'value': 'Toronto mayor goes into self-isolation after returning from U.K.'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://toronto.ctvnews.ca/toronto-mayor-goes-into-self-isolation-after-returning-from-u-k-1.4852470'}, {'href': 'https://www.ctvnews.ca/polopoly_fs/1.4852490.1584135682!/httpImage/image.jpg_gen/derivatives/landscape_300/image.jpg', 'rel': 'enclosure'}], 'link': 'https://toronto.ctvnews.ca/toronto-mayor-goes-into-self-isolation-after-returning-from-u-k-1.4852470', 'summary': 'Toronto Mayor John Tory will be going into self-isolation after returning from a business trip to London, England.', 'summary_detail': {'type': 'text/html', 'language': None, 'base': 'https://toronto.ctvnews.ca/rss/ctv-news-toronto-1.822319', 'value': 'Toronto Mayor John Tory wi

In [0]:

fp = urllib.request.urlopen("https://www.ontario.ca/page/2019-novel-coronavirus")
mybytes = fp.read()
soup = BeautifulSoup(mybytes, "lxml")  

mystr = soup.decode("utf8")
fp.close()

print(mystr)

<!DOCTYPE html>
<html class="no-js" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="2488e70214b0dfd8a40df37427e9633c8f527b3d" name="sha"/>
  <title>
  </title>
  <base href="/"/>
  <link href="/css/combined.css" media="screen" rel="stylesheet"/>
  <link href="/css/print.css" media="print" rel="stylesheet"/>
  <script src="/vendor/modernizr/modernizr.js">
  </script>
  <script>
   !function(a,e,o,n,c,i,t){a.GoogleAnalyticsObject=c,a.ga=a.ga||function(){(a.ga.q=a.ga.q||[]).push(arguments)},a.ga.l=1*new Date,i=e.createElement(o),t=e.getElementsByTagName(o)[0],i.async=1,i.src="//www.google-analytics.com/analytics.js",t.parentNode.insertBefore(i,t)}(window,document,"script",0,"ga"),ga("create","UA-21003310-6",{cookieDomain:"auto",allowLinker:!0}),ga("create","UA-48649528-1",{cookieDomain:"auto",allowLinker:!0,name:"ServiceOntario"}),ga("require","link